**NOTE**

This script MUST be run from the root / in order to work. Blame python for being idiotic, I don't know how to fix it

In [1]:
import h5py
import numpy as np
import pandas as pd
import selfies as sf
from tqdm import tqdm
import model.vae

In [2]:
vae = model.vae.load_vae_selfies()

/home/alden/Research/GDProject/model/vae/util.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path_to_vae_statedict, map_location=device)["state

loading model from saved_models/selfies_vae/selfies-vae.ckpt
Enc params: 1,994,592
Dec params: 277,346


In [3]:
# Create the SMILES, SELFIES csv file

smiles_dataset_path = "data/guacamol.smiles"
output_path = "data/molecule_dataset.csv"

def smiles_to_selfies(smiles: str):
    ''' Converts a smiles string to selfies representation - returns None on failure'''
    smiles = smiles.strip()
    try:
        selfies = sf.encoder(smiles=smiles, strict=True)
        return selfies
    except Exception as e:
        return None

# deprecated, don't use
# def create_smiles_selfies_csv(smiles_dataset_path="data/guacamol.smiles", output_path="data/molecule_dataset.csv"):
#     # get total lines
#     with open(smiles_dataset_path, 'r') as infile:
#         total_lines = sum(1 for line in infile if line.strip())
#         print(f"File has {total_lines} lines")
    
#     with open(smiles_dataset_path, 'r') as infile, open(output_path, 'w') as outfile:
#         outfile.write("SMILES,SELFIES\n")
        
#         for line in tqdm(infile, desc='Converting to SELFIES', total=total_lines):
#             smiles = line.strip()
#             if smiles:
#                 selfies = smiles_to_selfies(smiles)
#                 if selfies is not None:
#                     outfile.write(f"{smiles},{selfies}\n")
#                 else:
#                     print(f"Warning: Failed to convert SMILES '{smiles}' to SELFIES")

def create_smiles_selfies_h5(smiles_dataset_path="data/guacamol.smiles", output_path="data/molecule_dataset.h5"):
    # get total lines
    with open(smiles_dataset_path, 'r') as infile:
        total_lines = sum(1 for line in infile if line.strip())
    print(f"File has {total_lines} lines")
    
    with h5py.File(output_path, 'w') as h5file:
        # Create fixed-size datasets
        smiles_dataset = h5file.create_dataset('SMILES', (total_lines,), dtype=h5py.string_dtype())
        selfies_dataset = h5file.create_dataset('SELFIES', (total_lines,), dtype=h5py.string_dtype())
        
        idx = 0
        with open(smiles_dataset_path, 'r') as infile:
            for line in tqdm(infile, desc='Converting to SELFIES', total=total_lines):
                smiles = line.strip()
                if smiles:
                    selfies = smiles_to_selfies(smiles)
                    if selfies is not None:
                        smiles_dataset[idx] = smiles
                        selfies_dataset[idx] = selfies
                        idx += 1
                    else:
                        print(f"Warning: Failed to convert SMILES '{smiles}' to SELFIES")

In [ ]:
# create_smiles_selfies_h5()

File has 1591378 lines


Converting to SELFIES: 100%|██████████| 1591378/1591378 [08:08<00:00, 3257.52it/s]


In [4]:
def trim_molecule_dataset_h5(input_path="data/molecule_dataset.h5", output_path="data/molecule_dataset_trimmed.h5"):
    with h5py.File(input_path, 'r') as f_in:
        smiles = f_in['SMILES'][:]
        selfies = f_in['SELFIES'][:]
    
    valid_count = 0
    for i in range(len(smiles)):
        if smiles[i] and selfies[i]:
            valid_count += 1
        else:
            break
    
    with h5py.File(output_path, 'w') as f_out:
        f_out.create_dataset('SMILES', data=smiles[:valid_count])
        f_out.create_dataset('SELFIES', data=selfies[:valid_count])
    
    print(f"Trimmed from {len(smiles)} to {valid_count} elements")
    return valid_count

In [ ]:
# trim down dataset from the unused ones (the ones that couldnt convert smiles)
# trim_molecule_dataset_h5()

Trimmed from 1591378 to 1591344 elements


1591344

In [ ]:
# create_smiles_selfies_csv()
# creates the temporary SMILES, SELFIES pair dataset
# Takes ~5 mins

File as 1591378 lines


Converting to SELFIES: 100%|██████████| 1591378/1591378 [05:27<00:00, 4856.49it/s]


In [ ]:
import h5py
import numpy as np
import torch
from tqdm import tqdm

def process_selfies_to_latents(h5_path="data/molecule_dataset_trimmed.h5", vae=vae, batch_size=64, latent_dim=128, start_idx=0):
    """
    Process SELFIES to latents in batches, appending results progressively to H5 file.
    Maintains correspondence between SMILES[i] and LATENTS[i].
    """
    
    with h5py.File(h5_path, 'r+') as h5file:
        selfies_data = h5file['SELFIES']
        total_entries = len(selfies_data)
        
        # Create or get latents dataset
        if 'LATENTS' not in h5file:
            latents_dataset = h5file.create_dataset('LATENTS', (total_entries, latent_dim), dtype=np.float32)
        else:
            latents_dataset = h5file['LATENTS']
        
        # Process in batches
        for i in tqdm(range(start_idx, total_entries, batch_size)):
            batch_selfies = selfies_data[i:i+batch_size]
            batch_selfies = [selfie.decode('utf-8') for selfie in batch_selfies]
            
            try:
                # Batch processing
                batch_latents = model.vae.selfies_to_latent(selfies=batch_selfies, vae=vae)
                batch_latents = batch_latents.cpu().numpy()  # Shape: (batch_size, 128)
                
                latents_dataset[i:i+batch_size] = batch_latents

            except Exception as e:
               print(e)
               print(f"Failed to process batch at iteration {i}, returning")
               # Process each item individually

               for j, selfies in enumerate(batch_selfies):
                    idx = i + j
                    try:
                        z = model.vae.selfies_to_latent(selfies=selfies, vae=vae)
                        latents_dataset[idx] = z.cpu().numpy().squeeze()  # Remove batch dim
                    except Exception as individual_e:
                        print(f"Warning: Failed to process SELFIES at index {idx}: '{selfies}' - {individual_e}")
                        # Fill with zeros
                        latents_dataset[idx] = np.zeros(latent_dim, dtype=np.float32)
                    
            h5file.flush()

    print(f"Processing complete. Total entries: {total_entries}")


In [14]:
# # Deletes the latent space
# # do not run if you dont know what youre doing

# import h5py

# # Clear existing latent data
# with h5py.File("data/molecule_dataset_trimmed.h5", 'r+') as h5file:
#     if 'LATENTS' in h5file:
#         del h5file['LATENTS']
#         print("Deleted existing LATENTS dataset")
#     else:
#         print("No LATENTS dataset found")

In [ ]:
# process_selfies_to_latents()


  1%|          | 182/24865 [00:40<1:34:21,  4.36it/s]

'[=B-1]'
Failed to process batch at iteration 11648, returning


  2%|▏         | 560/24865 [02:07<1:37:56,  4.14it/s]

'[#S]'
Failed to process batch at iteration 35840, returning


  3%|▎         | 849/24865 [03:10<1:21:00,  4.94it/s]

...

'[P-1]'
Failed to process batch at iteration 459520, returning


 80%|████████  | 19997/24865 [1:13:07<17:39,  4.59it/s]

'[P-1]'
Failed to process batch at iteration 1279808, returning


100%|██████████| 24865/24865 [1:29:48<00:00,  4.61it/s]

Processing complete. Total entries: 1591344


In [16]:
with h5py.File("data/molecule_dataset_trimmed.h5", "r") as f:
    idx = 11669
    z = torch.tensor(f['LATENTS'][idx], device='cuda')
    print(z)
    print(model.vae.latent_to_selfies(z, vae=vae))
    print(f['SELFIES'][idx])

tensor([ 0.3796, -0.9044,  1.8945, -0.1288, -1.2595,  0.0097, -1.0654, -1.6163,
         0.7942,  0.0129, -0.1719, -1.1431,  0.6380, -1.1993, -1.0211, -0.5772,
         1.0090, -1.0820,  0.1975, -0.0310, -0.8889, -0.1467, -0.0071, -1.5427,
        -0.7854,  1.7676, -0.2289,  0.3882,  0.8806,  1.6932, -0.3385, -0.7470,
        -0.5508,  0.8731,  0.2137, -1.2020,  0.3163, -0.5795, -1.3604, -0.0808,
        -0.5355, -0.9419,  0.4012,  0.5543, -0.3302, -0.2996,  0.5294,  1.3599,
         0.0743, -0.3357,  1.0022, -0.4410,  0.2335, -1.4399,  0.2666,  0.2583,
         0.2118,  0.7277,  0.2814, -1.0767,  0.6510,  1.7454, -0.8206, -0.1665,
         0.4468, -0.5544, -2.0429,  1.5746, -0.6971, -0.6785, -0.0144, -0.7728,
        -0.8303,  0.5410,  0.4846, -1.2930,  0.4898, -0.2031,  0.0745,  0.4338,
        -0.6157,  0.1917,  0.5609, -0.2212,  0.3326,  1.6207, -0.0972,  1.0891,
         0.9262, -0.5864,  0.9220,  0.3795,  2.2645, -0.3452, -1.5416,  0.4260,
        -0.1082,  1.0440,  1.5560, -0.89

In [ ]:
def trim_molecule_dataset_final(input_path="data/molecule_dataset.h5", output_path="data/molecule_dataset_trimmed.h5"):
    with h5py.File(input_path, 'r') as f_in:
        smiles = f_in['SMILES'][:]
        selfies = f_in['SELFIES'][:]
        latents = f_in['LATENTS'][:]

        print(f"Len Smiles: {len(smiles)}")
        print(f"Len Selfies: {len(selfies)}")
        print(f"Len Latents: {len(latents)}")

        assert len(smiles) == len(selfies) == len(latents)
        dataset_len = len(smiles)

        invalid_latents = 0
        for latent in latents:
            if np.all(latent == 0.0):
                invalid_latents += 1
        
        total_lines = dataset_len - invalid_latents
        latent_dim = 128

        with h5py.File(output_path, 'w') as f_out:
            smiles_dataset = f_out.create_dataset('SMILES', (total_lines,), dtype=h5py.string_dtype())
            selfies_dataset = f_out.create_dataset('SELFIES', (total_lines,), dtype=h5py.string_dtype())
            latents_dataset = f_out.create_dataset('LATENTS', (total_lines, latent_dim), dtype=np.float32)

            idx = 0
            for i in tqdm(range(dataset_len)):
                smile = smiles[i]
                selfie = selfies[i]
                latent = latents[i]

                if np.all(latent == 0.0):
                    print(f"Skipping i: {i} with {smile} and latent {latent}")
                    continue

                smiles_dataset[idx] = smile
                selfies_dataset[idx] = selfie
                latents_dataset[idx] = latent
                idx += 1
                    

        
    # print(f"Trimmed from {len(smiles)} to {valid_count} elements")
    # return valid_count
# trim_molecule_dataset_final()

Len Smiles: 1591344
Len Selfies: 1591344
Len Latents: 1591344


  1%|          | 14256/1591344 [00:01<02:00, 13060.17it/s]

Skipping i: 11668 with b'CC[n+]1[b-](C(C)(C)C)n(N=C(C)C(C)(C)C)[b-](C(C)(C)C)[n+]1CC' and latent [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


  2%|▏         | 37679/1591344 [00:02<02:07, 12166.22it/s]

Skipping i: 35859 with b'N#CC(C#SN)=C1CCCCC1' and latent [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


...

 29%|██▉       | 461934/1591344 [00:37<01:30, 12533.78it/s]

Skipping i: 459560 with b'F[P-](F)(F)(F)(F)F' and latent [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


 81%|████████  | 1281715/1591344 [01:44<00:24, 12815.02it/s]

Skipping i: 1279836 with b'FC(F)(F)C(F)(F)[P-](F)(F)(F)(C(F)(F)C(F)(F)F)C(F)(F)C(F)(F)F' and latent [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


100%|██████████| 1591344/1591344 [02:08<00:00, 12359.52it/s]
